# **Tabela Vendedor**

Tratamento para que vendedores inativos mudem de nome.

In [1]:
CREATE OR ALTER PROCEDURE SP_CARREGA_VENDEDOR 
    @CODVENDEDOR SMALLINT,
    @VENDEDOR NVARCHAR(40),
    @BLOQUEIO CHAR(1),
    @CODSUPERVISOR SMALLINT,
    @SUPERVISOR NVARCHAR(40),
    @CODGERENTE SMALLINT,
    @GERENTE NVARCHAR(40),
    @CODAREA SMALLINT,
    @AREACOMERCIAL NVARCHAR(40)
AS
BEGIN
    DECLARE @DTUPDATE DATETIME = DATEADD(hour, -3, GETUTCDATE())
    SET @SUPERVISOR = CASE WHEN @SUPERVISOR = 'NULL' THEN NULL ELSE TRIM(@SUPERVISOR) END 
    SET @GERENTE = CASE WHEN @GERENTE = 'NULL' THEN NULL ELSE TRIM(@GERENTE) END 
    SET @AREACOMERCIAL = CASE WHEN @AREACOMERCIAL = 'NULL' THEN NULL ELSE TRIM(@AREACOMERCIAL) END 

    SET @VENDEDOR = 
        CASE WHEN (@BLOQUEIO = 'S' AND @CODSUPERVISOR = 1) THEN 'OUTROS CAPITAL'
             WHEN (@BLOQUEIO = 'S' AND @CODSUPERVISOR = 2) THEN 'OUTROS INTERIOR'
             WHEN (@BLOQUEIO = 'S' AND @CODGERENTE = 3) THEN 'OUTROS LOJA'
             WHEN (@BLOQUEIO = 'S' AND @CODSUPERVISOR = 4) THEN 'OUTROS MKT'
             WHEN @VENDEDOR = 'NULL' THEN NULL 
        ELSE TRIM(@VENDEDOR) END
    
    IF EXISTS (SELECT 1 FROM VENDEDOR WHERE CODVENDEDOR = @CODVENDEDOR)
    BEGIN
        UPDATE VENDEDOR 
        SET VENDEDOR = @VENDEDOR,
            CODSUPERVISOR = @CODSUPERVISOR,
            SUPERVISOR = @SUPERVISOR,
            CODGERENTE = @CODGERENTE,
            GERENTE = @GERENTE,
            CODAREA = @CODAREA,
            AREACOMERCIAL = @AREACOMERCIAL,
            DTUPDATE = @DTUPDATE 
        WHERE CODVENDEDOR = @CODVENDEDOR
    END
    ELSE
    BEGIN
        INSERT INTO VENDEDOR 
            (CODVENDEDOR, VENDEDOR, CODSUPERVISOR, SUPERVISOR, CODGERENTE, GERENTE, CODAREA, AREACOMERCIAL, DTUPDATE)
        VALUES 
            (@CODVENDEDOR, @VENDEDOR, @CODSUPERVISOR, @SUPERVISOR, @CODGERENTE, @GERENTE,@CODAREA, @AREACOMERCIAL, @DTUPDATE)
    END
END

Commands completed successfully.

Total execution time: 00:00:00.027

In [ ]:
CREATE OR ALTER PROCEDURE SP_RETORNA_VENDEDOR
    @CODVENDEDOR INT
AS
BEGIN
    IF EXISTS (SELECT 1 FROM VENDEDOR WHERE CODVENDEDOR = @CODVENDEDOR)
    BEGIN
        SELECT FORMAT(DTUPDATE, 'yyyy-MM-dd HH:mm:ss') DTUPDATE FROM VENDEDOR WHERE CODVENDEDOR = @CODVENDEDOR
    END
END

In [5]:
SELECT * FROM VENDEDOR